# 🛠️ Customer Complaint Resolver with Ollama + CrewAI

This notebook sets up a multi-agent system using Ollama and CrewAI to investigate a customer complaint and generate a resolution message.

In [ ]:
from crewai import Agent, Task, Crew
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.llms import Ollama
from crewai_tools import tool

In [ ]:
# Choose your model: 'llama3', 'phi', 'openhermes', etc.
ollama_llm = Ollama(model='llama3')

In [ ]:
@tool('DuckDuckGo Search')
def ddg_search(query: str) -> str:
    """Search the web for relevant information."""
    return DuckDuckGoSearchRun().run(query)

In [ ]:
# A mock complaint input
customer_complaint = "I ordered a product a week ago, but it still hasn’t arrived and customer service hasn’t responded to my emails."

In [ ]:
investigator = Agent(
    role='Complaint Investigator',
    goal='Find reasons behind common delivery issues and delays from customer complaints online.',
    backstory='You are an expert in logistics issues and customer support trends. You investigate complaints and gather facts.',
    verbose=True,
    allow_delegation=False,
    tools=[ddg_search],
    llm=ollama_llm
)

In [ ]:
resolver = Agent(
    role='Resolution Specialist',
    goal='Craft professional, helpful replies to customers that acknowledge the issue and propose next steps.',
    backstory='You are a friendly and empathetic customer support rep known for turning bad experiences into positive ones.',
    verbose=True,
    allow_delegation=True,
    llm=ollama_llm
)

In [ ]:
investigate_task = Task(
    description=f"Investigate the root causes of the following complaint: '{customer_complaint}'",
    agent=investigator
)

resolve_task = Task(
    description="Write a customer support response that addresses the problem found in the investigation and offers a helpful resolution.",
    agent=resolver,
    depends_on=[investigate_task]
)

In [ ]:
crew = Crew(
    agents=[investigator, resolver],
    tasks=[investigate_task, resolve_task],
    verbose=True
)

output = crew.run()
print("\n--- Final Output ---\n")
print(output)